In [1]:
import pandas as pd

In [94]:
year = 2020

afc = pd.read_csv(str(year)+'/AFC.csv')
afc = afc[~afc['Tm'].str.contains('AFC')]
afc.insert(2, 'Conference', 'AFC')
afc.insert(4, 'Division', ['East']*4 + ['North']*4 + ['South']*4 + ['West']*4)
afc = afc[['Tm', 'Conference', 'Division']].rename(columns={'Tm': 'Current_Name'})

nfc = pd.read_csv(str(year)+'/NFC.csv')
nfc = nfc[~nfc['Tm'].str.contains('NFC')]
nfc.insert(2, 'Conference', 'NFC')
nfc.insert(4, 'Division', ['East']*4 + ['North']*4 + ['South']*4 + ['West']*4)
nfc = nfc[['Tm', 'Conference', 'Division']].rename(columns={'Tm': 'Current_Name'})

teams = pd.concat([afc, nfc]).reset_index()[['Current_Name', 'Conference', 'Division']]
teams['Current_Name'] = teams['Current_Name'].apply(lambda s: s.rstrip('*+ '))
teams.insert(0, 'team_id', teams.index)

# This one probably won't be as useful
index_to_team = dict(zip(teams.index, teams['Current_Name']))

# Useful Mapping
team_to_index = dict(zip(teams['Current_Name'], teams.index))
team_to_index.update({"Washington Redskins": 16, "Oakland Raiders": 13, "San Diego Chargers": 15, "St. Louis Rams": 29})

print(teams)
print(index_to_team)
print(team_to_index)

    team_id              Current_Name Conference Division
0         0             Buffalo Bills        AFC     East
1         1            Miami Dolphins        AFC     East
2         2      New England Patriots        AFC     East
3         3             New York Jets        AFC     East
4         4       Pittsburgh Steelers        AFC    North
5         5          Cleveland Browns        AFC    North
6         6          Baltimore Ravens        AFC    North
7         7        Cincinnati Bengals        AFC    North
8         8          Tennessee Titans        AFC    South
9         9        Indianapolis Colts        AFC    South
10       10            Houston Texans        AFC    South
11       11      Jacksonville Jaguars        AFC    South
12       12        Kansas City Chiefs        AFC     West
13       13         Las Vegas Raiders        AFC     West
14       14            Denver Broncos        AFC     West
15       15      Los Angeles Chargers        AFC     West
16       16  W

In [117]:
season_team_stats = None
for year in range(2002, 2021):
    afc = pd.read_csv(str(year)+'/AFC.csv')
    afc = afc[~afc['Tm'].str.contains('AFC')]

    nfc = pd.read_csv(str(year)+'/NFC.csv')
    nfc = nfc[~nfc['Tm'].str.contains('NFC')]
    
    teams = afc.append(nfc)
    teams['Tm'] = teams['Tm'].apply(lambda s: s.rstrip('*+ '))
    teams.insert(0, 'team_id', [team_to_index[team] for team in teams['Tm']])
    teams.insert(1, 'year', year)
    teams = teams.rename(columns={'W': 'wins', "L": "losses", "T": "ties", "PF": "points_for",
                          "PA": "points_against", "SoS": "strength_of_schedule"})
    if 'ties' not in teams.columns:
        teams['ties'] = 0
        
    teams = teams[['team_id', 'year', 'wins', 'losses', 'ties', 'points_for', 'points_against', 'strength_of_schedule']]
    
    if season_team_stats is None:
        season_team_stats = teams
    else:
        season_team_stats = season_team_stats.append(teams)
        
season_team_stats = season_team_stats.reset_index()
print(season_team_stats)

     index  team_id  year  wins  losses  ties  points_for  points_against  \
0        1        3  2002   9.0     7.0   0.0       359.0           336.0   
1        2        2  2002   9.0     7.0   0.0       381.0           346.0   
2        3        1  2002   9.0     7.0   0.0       378.0           301.0   
3        4        0  2002   8.0     8.0   0.0       379.0           397.0   
4        6        4  2002  10.0     5.0   1.0       390.0           345.0   
..     ...      ...   ...   ...     ...   ...         ...             ...   
603     14       27  2020   4.0     8.0   0.0       280.0           300.0   
604     16       28  2020   8.0     3.0   0.0       341.0           304.0   
605     17       29  2020   7.0     4.0   0.0       263.0           215.0   
606     18       30  2020   6.0     5.0   0.0       304.0           258.0   
607     19       31  2020   5.0     6.0   0.0       261.0           254.0   

     strength_of_schedule  
0                     1.7  
1                  